In [1]:
import numpy as np 
import pandas as pd

In [16]:
# import train csv
df_train = pd.read_csv('/Users/ahmedehab/Desktop/machine learning project/titanic_prediction/Data/train.csv')

# import test
df_test = pd.read_csv('/Users/ahmedehab/Desktop/machine learning project/titanic_prediction/Data/test.csv')

df = pd.concat([df_train, df_test], ignore_index=True)


In [17]:
# validate imported CSVs
print(df_train.shape)
print(df_test.shape)
print(df.shape)


(891, 12)
(418, 11)
(1309, 12)


In [4]:
# exploring df_train

In [18]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# the survival is encoded to 0 ,1
# i will need to remove the passenger ID
# i will extract the title from the name as it will make the data more rich with info (try to dodge overfitting) and then drop name col
# i will encode Sex column to male --> 0 , female --> 1
# i will standarize the age 
# i do not know what should i do with the Ticket and fare 
# i will use one hot encoding for the embarked 

In [19]:
# getting the info about the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [8]:
# it seems fine 

In [20]:
# investigating the null values
df.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [10]:
# Age            177
# Cabin          687

# i will use the median to impute the age with the median or extracting most correlated feature to the age 
# and use pred model to impute it .

# for the cabin i will use mode 

In [21]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,891.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,0.383838,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.486592,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


# Feature Engineering Recon Notes

- The `Survived` column is already encoded as 0 and 1, so no need to change anything there.
- I’ll drop the `PassengerId` column — it’s just a unique ID and won’t help with predictions.
- I plan to extract the title (like Mr, Miss, etc.) from the `Name` column to enrich the data a bit. Once that's done, I’ll drop the original `Name`.
- I’ll encode the `Sex` column: male as 0, female as 1.
- For `Age`, I’ll probably fill in the missing values using the median grouped by title and then standardize it.
- Not sure yet what to do with `Ticket` and `Fare`. Might explore extracting something from the ticket or handling outliers in fare later.
- For `Embarked`, I’ll fill the missing values (probably with the most common one) and use one-hot encoding since it’s categorical.

Just mapping things out for now — no preprocessing yet.


In [12]:
# data preprocessing

In [22]:
# creating list for dropped columns
columns_drop = ['PassengerId','Ticket']

# dropping columns
df.drop(columns_drop,axis=1,inplace=True)

# validate
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
# extract title from Name